In [129]:
from mpi4py import MPI

In [130]:
from struphy.geometry.domains import Cuboid

l1 = -.5
r1 = .5
l2 = -.5
r2 = .5
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.pic.particles import ParticlesSPH

# mandatory parameters
name = 'test'
Np = 1000
bc = ['reflect', 'reflect', 'periodic']
loading = 'pseudo_random'

# optional
loading_params = {'seed': None}
params_sorting = {"nx": 1, "ny": 1, "nz": 1, "eps": 0.25}
params_loading = {"seed": 1234, "moments": "degenerate", "spatial": "uniform"}

bckgr_params = {
        "type": "BeltramiFlow",
        "BeltramiFlow": {},
        "pforms": [None, None],
    }

mpi_comm = MPI.COMM_WORLD

# instantiate Particle object
particles = ParticlesSPH(
        name=name,
        Np=Np,
        bc=bc,
        loading=loading,
        eps=10.0,  # Lots a buffering needed since only 3*3*3 box
        comm= mpi_comm,
        loading_params=params_loading,
        domain=domain,
        bckgr_params=bckgr_params,
        sorting_params=params_sorting,
    )


In [ ]:
particles.draw_markers(sort=False)
particles.mpi_sort_markers()
particles.initialize_weights()

In [ ]:
particles.positions
# positions on the physical domain Omega
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=False)
print(opts_eta)

In [135]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [136]:
# instantiate Propagator object
prop_eta = PushEta(particles, algo = "forward_euler")

In [137]:
from struphy.feec.psydac_derham import Derham

Nel = [64, 64, 1]  # Number of grid cells
p = [3, 3, 1]  # spline degrees
spl_kind = [False, False, True]   # spline types (clamped vs. periodic)

derham = Derham(Nel, p, spl_kind)

In [138]:
from struphy.propagators.propagators_markers import PushVinEfield

# instantiate Propagator object
PushVinEfield.domain = domain
PushVinEfield.derham = derham

In [ ]:
from struphy.fields_background.fluid_equils.equils import BeltramiFlow
bel_flow = BeltramiFlow()
p_xyz = bel_flow.p_xyz
p = lambda eta1, eta2, eta3: domain.pull(p_xyz, eta1, eta2, eta3)
p_coeffs = derham.P["0"](p)
p = lambda eta1, eta2, eta3: domain.pull(p_xyz, eta1, eta2, eta3, squeeze_out=True)
p_coeffs

In [140]:
p_h = derham.create_field('pressure', 'H1')
p_h.vector = p_coeffs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 12))
x = np.linspace(-.5, .5, 100)
y = np.linspace(-.5, .5, 90)
xx, yy = np.meshgrid(x, y)
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)

plt.subplot(2, 2, 1)
plt.pcolor(xx, yy, p_xyz(xx, yy, 0))
plt.axis('square')
plt.title('p_xyz')
plt.colorbar()

plt.subplot(2, 2, 2)
p_vals = p(eta1, eta2, 0).T
plt.pcolor(eta1, eta2, p_vals)
plt.axis('square')
plt.title('p logical')
plt.colorbar()

plt.subplot(2, 2, 3)
p_h_vals = p_h(eta1, eta2, 0, squeeze_out=True).T
plt.pcolor(eta1, eta2, p_h_vals)
plt.axis('square')
plt.title('p_h (logical)')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.pcolor(eta1, eta2, np.abs(p_vals - p_h_vals))
plt.axis('square')
plt.title('difference')
plt.colorbar()

In [142]:
grad_p_ana = lambda x, y: -1.*np.array([np.pi * np.sin(np.pi*x)*np.cos(np.pi*x),
                             np.pi * np.sin(np.pi*y)*np.cos(np.pi*y)])

In [143]:
grad_p = derham.grad.dot(p_coeffs)
grad_p.update_ghost_regions() # very important, we will move it inside grad
grad_p *= -1.
prop_v = PushVinEfield(particles, e_field=grad_p)

In [144]:
grad_p_h = derham.create_field('pressure gradient', 'Hcurl')
grad_p_h.vector = grad_p

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 18))

plt.subplot(3, 2, 1)
plt.pcolor(xx, yy, grad_p_ana(xx, yy)[0])
plt.axis('square')
plt.title('grad_p x')
plt.colorbar()

plt.subplot(3, 2, 2)
plt.pcolor(xx, yy, grad_p_ana(xx, yy)[1])
plt.axis('square')
plt.title('grad_p y')
plt.colorbar()

plt.subplot(3, 2, 3)
grad_p_vals = grad_p_h(eta1, eta2, 0, squeeze_out=True)
plt.pcolor(eta1, eta2, grad_p_vals[0].T)
plt.axis('square')
plt.title('grad_p_h x')
plt.colorbar()

plt.subplot(3, 2, 4)
plt.pcolor(eta1, eta2, grad_p_vals[1].T)
plt.axis('square')
plt.title('grad_p_h x')
plt.colorbar()

plt.subplot(3, 2, 5)
plt.pcolor(eta1, eta2, np.abs(grad_p_vals[0].T - grad_p_ana(xx, yy)[0]))
plt.axis('square')
plt.title('diff x')
plt.colorbar()

plt.subplot(3, 2, 6)
plt.pcolor(eta1, eta2, np.abs(grad_p_vals[1].T - grad_p_ana(xx, yy)[1]))
plt.axis('square')
plt.title('diff y')
plt.colorbar()

In [ ]:
import numpy as np
import math
import tqdm

# time stepping
dt = 0.02
Nt = 200

pos = np.zeros((Nt + 1, Np, 3), dtype=float)
velo = np.zeros((Nt + 1, Np, 3), dtype=float)
energy = np.zeros((Nt + 1, Np), dtype=float)

particles.draw_markers(sort=False)
particles.mpi_sort_markers()
particles.initialize_weights()

pos[0] = domain(particles.positions).T
velo[0] = particles.velocities
energy[0] = .5*(velo[0, : , 0]**2 + velo[0, : , 1]**2) + p_h(particles.positions)

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0
while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta(dt/2)
    prop_v(dt)
    prop_eta(dt/2)
    
    # positions on the physical domain Omega
    pos[n] = domain(particles.positions).T
    velo[n] = particles.velocities
    
    energy[n] = .5*(velo[n, : , 0]**2 + velo[n, : , 1]**2) + p_h(particles.positions)

In [ ]:
# energy plots
fig = plt.figure(figsize = (13, 6))

plt.subplot(2, 2, 1)
plt.plot(time_vec, energy[:, 0])
plt.title('particle 1')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 2)
plt.plot(time_vec, energy[:, 1])
plt.title('particle 2')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 3)
plt.plot(time_vec, energy[:, 2])
plt.title('particle 3')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 4)
plt.plot(time_vec, energy[:, 3])
plt.title('particle 4')
plt.xlabel('time')
plt.ylabel('energy')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter(pos[-1,:,0],pos[-1,:,1],pos[-1,:,2])

In [ ]:
import matplotlib.animation as animation
n_frame = 200
fig, ax = plt.subplots()

coloring = np.select([pos[0,:,0]<=-0.2, np.abs(pos[0,:,0]) < +0.2, pos[0,:,0] >= 0.2],
                     [-1.0, 0.0, +1.0])
scat = ax.scatter(pos[0,:,0], pos[0,:,1], c=coloring)
ax.set_xlim([-0.5,0.5])
ax.set_ylim([-0.5,0.5])
ax.set_aspect('equal')

f = lambda x, y: np.cos(np.pi*x)*np.cos(np.pi*y)
ax.contour(xx, yy, f(xx, yy))
ax.set_title(f'time = {time_vec[0]:4.2f}')

def update_frame(frame):
    scat.set_offsets(pos[frame,:,:2])
    ax.set_title(f'time = {time_vec[frame]:4.2f}')
    return scat

ani = animation.FuncAnimation(fig=fig, func=update_frame, frames = n_frame)
ani.save("Beltramiaktuell.gif")

In [ ]:
import matplotlib.animation as animation
n_frame = 10
fig, ax = plt.subplots()

coloring = np.select([pos[0]<=-0.2, np.abs(pos[0]) < +0.2, pos[0] >= 0.2],
                     [-1.0, 0.0, +1.0])
scat = ax.scatter(pos[0], pos[1])
def update_frame(frame):
    scat.set_offsets(pos[frame])
    return scat

ani = animation.FuncAnimation(fig=fig, func=update_frame, frames = n_frame)
ani.save("Beltramiaktuell2.gif")
np.shape(pos[0])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')

coloring = np.select([pos[0]<=-0.2, np.abs(pos[0]) < +0.2, pos[0] >= 0.2],
                     [-1.0, 0.0, +1.0])
scatter = ax.scatter(pos[0], pos[1])
ax.set_xlim([-0.5, +0.5])
ax.set_ylim([-0.5, +0.5])
def animate(i):
    pos[i]
    ##scatter.set_offsets(Xi.T)
 
    anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=1,
                               repeat_delay=5000) 
plt.show() 

##anim.save("test.gif", writer='pillow', fps=60)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation

t = np.linspace(0,10,100)
y = np.sin(t)

fig, axis = plt.subplots()
axis.set_xlim([min(t), max(t)])
axis.set_ylim([-2,2])
animated_plot, = axis.plot([],[])

def update_data(frame):
    animated_plot.set_data(t,y)
    return animated_plot

animation = FuncAnimation(fig=fig, func=update_data, frames = len(t), interval = 0.01)

plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Create time and y data
t = np.linspace(0, 10, 100)
y = np.sin(t)

# Create a figure and an axis
fig, axis = plt.subplots()
axis.set_xlim([min(t), max(t)])
axis.set_ylim([-2, 2])

# Initialize the plot with empty data
animated_plot, = axis.plot([], [], lw=2)

# Update function to animate the plot
def update_data(frame):
    # Set the data for the current frame
    animated_plot.set_data(t[:frame], y[:frame])  # Display data up to the current frame
    return animated_plot,  # Return the plot object (blit requires an iterable)

# Create the animation
animation = FuncAnimation(fig, update_data, frames=len(t), interval=25, blit=True)

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


#--- Parameters
N = 1            # number of particles per direction
dt = 1./50.       # time step
n_steps = 10 ###10000      # max. number of steps
x_offset = 0.05   # off-set at the boundary


#--- Velocity field
u = lambda x: np.array([-np.cos(np.pi*x[0])*np.sin(np.pi*x[1]),
                         np.sin(np.pi*x[0])*np.cos(np.pi*x[1])])

#--- Pressure field
p = lambda x: 0.5*(np.sin(np.pi*x[0])**2 + np.sin(np.pi*x[1])**2)

#--- Analytical pressure gradient
grad_p = lambda x: np.array([np.pi * np.sin(np.pi*x[0])*np.cos(np.pi*x[0]),
                             np.pi * np.sin(np.pi*x[1])*np.cos(np.pi*x[1])])

#--- Perturbed analytical pressure gradient
# grad_p = lambda x: np.array([np.pi * np.sin(np.pi*x[0])*np.cos(np.pi*x[0]) + 0.1*np.random.random(),
#                              np.pi * np.sin(np.pi*x[1])*np.cos(np.pi*x[1]) + 0.1*np.random.random()]) 

#--- Invariant and Hamiltonian
f = lambda x: np.cos(np.pi*x[0])*np.cos(np.pi*x[1])
k = lambda v: 0.5*(v[0]**2 + v[1]**2)
h = lambda x, v: 0.5*(v[0]**2 + v[1]**2) + p(x)

#--- Update function for the Lagrangian trajectory
#    Hamiltonian system where p acts as a potential energy
#    Symplectic Euler scheme
def update(x, v, dt):
    """
    Symplectic Euler for separable Hamiltonians.
    """

    # Classical symplectic Euler
    # (equivalent to Hamiltonian Lie splitting)
    # x += dt * v
    # v -= dt * grad_p(x)

    # Hamiltonian Strang splitting
    # x += 0.5 * dt * v
    print(x + .5)
    print(dt, -grad_p(x))
    v -= dt * grad_p(x)
    # x += 0.5 * dt * v 
        
    return (x,v)

#--- Initialization of the particles on a uniform grid
# dx = x_offset
# s = np.linspace(-0.5+dx, +0.5-dx, N)
# XX, YY = np.meshgrid(s, s)
# X0 = XX.flatten()
# X1 = YY.flatten()
# X = np.row_stack((X0,X1))

#--- Random initialization of particles
np.random.seed(1234)
X = -0.5 + np.random.random((2,N*N))

print(X)

#--- Initialization of velocities
V = u(X)

print(V)

Lagr_trajectories = np.empty(X.shape+(n_steps,))
Lagr_velocities = np.empty(V.shape+(n_steps,))
Lagr_trajectories[...,0] = X
Lagr_velocities[...,0] = V
for i in range(1,n_steps):
    X, V = update(X, V, dt)
    Lagr_trajectories[...,i] = X
    Lagr_velocities[...,i] = V

print('Computation completed. Plotting...')
   
#--- Finer grid for the contours of the invariant
Sg = np.linspace(-0.5, +0.5, 200)
Xg, Yg = np.meshgrid(Sg, Sg)

#--- Animation
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')
ax.contour(Xg, Yg, f([Xg,Yg]))
Xi = Lagr_trajectories[...,0]
coloring = np.select([Xi[0]<=-0.2, np.abs(Xi[0]) < +0.2, Xi[0] >= 0.2],
                     [-1.0, 0.0, +1.0])
scatter = ax.scatter(Xi[0], Xi[1], c=coloring, marker='.', cmap='rainbow')
ax.set_xlim([-0.5, +0.5])
ax.set_ylim([-0.5, +0.5])
def animate(i):
    Xi = Lagr_trajectories[...,i]
    scatter.set_offsets(Xi.T)
    
anim = animation.FuncAnimation(fig, animate, frames=n_steps, interval=1,
                               repeat_delay=5000) 

#--- Hamiltonian, invariant, and calculation of the total energies
figh = plt.figure()
axh1 = figh.add_subplot(121)
axh2 = figh.add_subplot(122)
tot_Ham = np.zeros_like(Lagr_trajectories[0,0,:])
tot_Kin = np.zeros_like(Lagr_trajectories[0,0,:])
tot_Ein = np.zeros_like(Lagr_trajectories[0,0,:])
for ip in range(Lagr_trajectories.shape[1]):
    particle_h = h(Lagr_trajectories[:,ip,:], Lagr_velocities[:,ip,:])
    particle_k = k(Lagr_velocities[:,ip,:])
    particle_p = p(Lagr_trajectories[:,ip,:])
    particle_f = f(Lagr_trajectories[:,ip,:])
    tot_Ham += particle_h
    tot_Kin += particle_k
    tot_Ein += particle_p
    if ip < 100: # only for the first 100 particles
        axh1.plot(particle_h[:] - particle_h[0])
        axh2.plot(particle_f[:] - particle_f[0])

#--- Normalization to the number of particles        
tot_Ham /= N*N
tot_Kin /= N*N
tot_Ein /= N*N

print('... h and total h computed and plotted ...')
print('Initial kinetic energy = {}'.format(tot_Kin[0]))
print('Initial potential energy = {}'.format(tot_Ein[0]))
print('Initial Hamiltonian = {}'.format(tot_Ham[0]))

#--- Total Hamiltonian conservation error
figth = plt.figure()
axth1 = figth.add_subplot(121)
axth1.plot(tot_Ham[:] - tot_Ham[0])
axth2 = figth.add_subplot(122)
axth2.plot(tot_Kin[:] - tot_Kin[0])
axth2.plot(tot_Ein[:] - tot_Ein[0])


plt.show() 

###anim.save("test.gif", writer='pillow', fps=60)

In [154]:
X = -0.5 + np.random.random((2,N*N))

In [155]:
N= 2

In [156]:
X =  np.random.random((2,3))

In [ ]:
X


In [ ]:
X.shape

In [159]:
Lagr_trajectories = np.empty(X.shape+(n_steps,))